# これなに？


## COOの性質
TODO

# やり方

## バージョン

## ダミー化する

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
from scipy.sparse import coo_matrix 
from seaborn.utils import load_dataset

# ダミー化したいデータ
x=load_dataset('titanic')['embark_town']
x=x.fillna('NaN')

# 見やすくするためDataFrameに変換
pd.DataFrame(x.head())

,embark_town
0,Southampton
1,Cherbourg
2,Southampton
3,Southampton
4,Southampton


In [2]:
x_index,category=pd.factorize(x,sort=True) # 追加でdummy化するデータと整合性を保つためsortしておく
x_index[:10]

array([3, 0, 3, 3, 3, 2, 3, 3, 3, 0])

In [3]:
category

Index(['Cherbourg', 'NaN', 'Queenstown', 'Southampton'], dtype='object')

In [4]:
x_size=len(x_index)
row=np.arange(x_size)
x_coo=coo_matrix((np.broadcast_to(1,x_size),(row,x_index)))

In [5]:
x_coo.toarray()[:10]

array([[0, 0, 0, 1],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [1, 0, 0, 0]])

## データを後で追加する

追加するデータを作成

In [6]:
add=np.random.choice(category,100)
add[:10]

array(['Queenstown', 'Cherbourg', 'Southampton', 'Southampton',
       'Queenstown', 'Queenstown', 'NaN', 'Queenstown', 'Queenstown',
       'NaN'], dtype=object)

同じようにダミー化

In [7]:
add_index,_=pd.factorize(add,sort=True)

In [8]:
add_size=len(add_index)
add_row=np.arange(add_size)
add_coo=coo_matrix((np.broadcast_to(1,add_size),(add_row,add_index)))

In [9]:
add_coo.toarray()[:10]

array([[0, 0, 1, 0],
       [1, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 1, 0, 0]])

結合する

## pandas.get_dummiesを使用した場合と結果が同じか確認

In [119]:
all_dummied=pd.get_dummies(np.concatenate((x,add)))                      
(all_coo.toarray()==all_dummied.values).all()

True

全く同じ結果になる

ちなみに、`get_dummies`のオプションに`sparse`がありますが、これはTODO

# パフォーマンス

巨大なデータセットを作成

In [96]:
import itertools as it
test_data=np.random.choice(x,100000)
test_data.shape

(100000,)

## 実行時間

### pandas.get_dummiesを使用した場合

In [97]:
%timeit pd.get_dummies(test_data)

5.28 ms ± 119 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


### pandas.get_dummies(sparse=True)を使用した場合

In [120]:
%timeit pd.get_dummies(test_data,sparse=True)

268 ms ± 6.87 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


更に遅くなりました

### sparse.coo_matrixを応用する場合

In [98]:
%%timeit
x_index,_=pd.factorize(x,sort=True)
x_size=len(x_index)
row=np.arange(x_size)
x_coo=coo_matrix((np.broadcast_to(1,x_size),(row,x_index)))

364 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


圧倒的に`coo`形式で変換したほうが早いです。
ちなみに、`get_duumies`のオプションの`spase`を`True`した場合、TODO

# TODO 以下はあまり変わらなかったので、toolzの内容を一緒にしたほうが良いかも

## メモリ

`memory_profiler`を使用して計測します

In [99]:
!pip install memory_profiler

You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


マジックコマンドが適用できるようにロードします

In [100]:
 %load_ext memory_profiler

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler


### pandas.get_dummiesを使用した場合

In [118]:
%mprun -f pd.get_dummies pd.get_dummies(test_data)

`sprse=True`にしても変わりません

In [112]:
%mprun -f pd.get_dummies  pd.get_dummies(test_data,sparse=True)

### sparse.coo_matrixを応用する場合

計測したいコードが複数行にまたがる場合、一度ファイルに書き出す必要がります。

In [107]:
%%file dummies_perfomance.py
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
def dummies_coo(x):
    x_index,_=pd.factorize(x,sort=True)
    x_size=len(x_index)
    row=np.arange(x_size)
    x_coo=coo_matrix((np.broadcast_to(1,x_size),(row,x_index)))
    

Overwriting dummies_perfomance.py


In [114]:
%reload_ext autoreload
%autoreload 2

In [113]:
from dummies_perfomance import dummies_coo
%mprun -f dummies_coo  dummies_coo(test_data)

# toolzを応用してストリーミング形式で変換していく

記事分けるかも